In [39]:
import sys
from pyspark import SparkConf, SparkContext
from operator import itemgetter
conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)
lines = sc.textFile("browsing.txt")

products = lines.flatMap(lambda x: x.split(" ")).countByValue()
pro_100={}
for key,value in products.items():#single element which appears more than 100
    if value>=100:
         pro_100.setdefault(str(key),int(value))
            
def split_line(line):#split every line in to separate elements
    pros=[]
    for pro in line.split():
        pros.append(str(pro))
    return pros
def pair2_product(p):
    pairs=[]
    n=len(p)
    for i in range(0,n-1):
        for j in range(i+1,n):
            if (p[i] in pro_100) and (p[j] in pro_100):
                pairs.append(((p[i],p[j]),1))
                pairs.append(((p[j],p[i]),1))
    return pairs

pro=lines.map(split_line)
p_pairs=pro.flatMap(pair2_product)
counted=p_pairs.groupByKey() \
    .map(lambda x: (x[0], sum(x[1])))\
    .filter(lambda x: x[1] >= 100) 
counted=counted.collect()
key=[]
value=[]
for i in counted:
    key.append(i[0])
    value.append(i[1])
pair_100=zip(key,value)
pair_100=dict(pair_100)

def pair3_product(p):
    pairs=[]
    n=len(p)
    for i in range(0,n-2):
        for j in range(i+1,n-1):
            for k in range(j+1,n):
                #if (((p[i],p[j]) in pair_100) and (p[k] in pro_100)):
                #    pairs.append((((p[i],p[j]),p[k]),1))
                #if (((p[j],p[k]) in pair_100) and (p[i] in pro_100)):
                #    pairs.append((((p[j],p[k]),p[i]),1))
                #if (((p[j],p[k]) in pair_100) and (p[i] in pro_100)):
                #    pairs.append((((p[i],p[k]),p[j]),1))
                if (((p[i],p[j]) in pair_100) and (p[k] in pro_100)) or (((p[i],p[k]) in pair_100) and (p[j] in pro_100)) or (((p[j],p[k]) in pair_100) and (p[i] in pro_100)):
                    pairs.append(((p[i],p[j],p[k]),1))
                    pairs.append(((p[i],p[k],p[j]),1))
                    pairs.append(((p[j],p[i],p[k]),1))
                    pairs.append(((p[j],p[k],p[i]),1))
                    pairs.append(((p[k],p[j],p[i]),1))
                    pairs.append(((p[k],p[i],p[j]),1))
    return pairs
def confidence(p):
    sup1=int(pair_100[(p[0][0],p[0][1])])
    conf1=p[1]/sup1
    sup2=int(pair_100[(p[0][0],p[0][2])])
    conf2=p[1]/sup2
    sup3=int(pair_100[(p[0][1],p[0][2])])
    conf3=p[1]/sup3
    return ((((p[0][0],p[0][1]),p[0][2]),conf1),(((p[0][0],p[0][2]),p[0][1]),conf2),(((p[0][1],p[0][2]),p[0][0]),conf3))

p_pairs3=pro.flatMap(pair3_product)
#counted2=p_pairs2.groupByKey() \#count(((pi,pj),pk),1)
#    .filter(lambda x: 2 not in x[1]) \
#    .map(lambda x: (x[0], sum(x[1])))\
#    .filter(lambda x: x[1] >= 100)
counted3=p_pairs3.groupByKey() \
    .map(lambda x: (x[0], sum(x[1])))\
    .filter(lambda x: x[1] >= 100)
result=counted3.flatMap(confidence)
#result.saveAsTextFile("result5.txt")
result=result.collect()
result=list(set(result))
result=sorted(result,key=lambda x: (-x[1], x[0][0][0],x[0][0][1],x[0][1]))
print(result[0:5])
sc.stop()

[((('DAI23334', 'ELE92920'), 'DAI62779'), 1.0), ((('DAI31081', 'GRO85051'), 'FRO40251'), 1.0), ((('DAI55911', 'GRO85051'), 'FRO40251'), 1.0), ((('DAI62779', 'DAI88079'), 'FRO40251'), 1.0), ((('DAI75645', 'GRO85051'), 'FRO40251'), 1.0)]
